# Voting

In [1]:
# Importing the libraries
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

# Importing the dataset
dataset = pd.read_csv('CVD dataset2.csv')
data=dataset.iloc[:, :].values
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, 16].values
# X = dataset.iloc[:, [ 0, 1, 2, 5, 10, 11, 13, 15]].values

rows = len(data)    # 3 rows in your example
cols = len(data[0])
print(rows)
print(cols)

560
17


In [2]:
dataNoCvd = data[~(data[:,16] > 0.0)]
dataCvd = data[~(data[:,16] < 1.0)]
rowsNoCvd = len(dataNoCvd)    
rowsCvd = len(dataCvd)
print(rowsNoCvd)
print(rowsCvd)
# print(dataNoCvd)
# print(dataCvd)

519
41


In [3]:
def find_NoCvd_limitsArray(i,DownNoCvd, UpNoCvd, fold_NoCvd_total, fold_NoCvd_residue):
    if i>0 :
        DownNoCvd = UpNoCvd
        UpNoCvd= fold_NoCvd_total+UpNoCvd
    if i < fold_NoCvd_residue :
        UpNoCvd+= 1
#     print("NoCvd limits, Down :",DownNoCvd,"Up ",UpNoCvd)
    return   DownNoCvd, UpNoCvd   
 
def find_Cvd_limitsArray(i,DownCvd, UpCvd, fold_Cvd_total, fold_Cvd_residue,cv):
    if i>0 :
        DownCvd = UpCvd
        UpCvd= fold_Cvd_total+UpCvd
    if i >= cv - fold_Cvd_residue :
         UpCvd+= 1     
#     print("Cvd limits, Down :",DownCvd,"Up ",UpCvd)
    return   DownCvd, UpCvd  

def find_testValSubset(DownNoCvd,UpNoCvd,DownCvd,UpCvd):
    temp1=dataNoCvd[DownNoCvd:UpNoCvd,:]
    temp2=dataCvd[DownCvd:UpCvd,:]
    temp3=np.concatenate((temp1, temp2))
#     print(len(temp1),"  ",len(temp2),"  ",len(temp3))
    return temp3

def find_trainSubset(DownNoCvd,UpNoCvd,DownCvd,UpCvd):
    temp1 = np.delete(dataNoCvd, slice(DownNoCvd, UpNoCvd), axis=0)
    temp2 = np.delete(dataCvd, slice(DownCvd, UpCvd), axis=0)
    temp3 = np.concatenate((temp1, temp2))
#     print(len(temp1),"  ",len(temp2),"  ",len(temp3))
    return temp1, temp2, temp3

In [4]:
def create_train_test_sets(cv,dataNoCvd,dataCvd,val_ratio):
    test_total = []
    train_total = []
    train_total_Cvd = []
    train_total_NoCvd = []
    validation_total = []
    
    rowsNoCvd = len(dataNoCvd)    
    rowsCvd = len(dataCvd)

    fold_Cvd_total = rowsCvd//cv
    fold_Cvd_residue= rowsCvd%cv
    print("fold_Cvd_total  :",fold_Cvd_total," fold_Cvd_residue  :",fold_Cvd_residue)

    fold_NoCvd_total = rowsNoCvd//cv
    fold_NoCvd_residue= rowsNoCvd%cv
    print("fold_NoCvd_total:",fold_NoCvd_total,"fold_NoCvd_residue:",fold_NoCvd_residue)
    
    Cvd_val = round(504*val_ratio*(rowsCvd/len(data)))
    noCvd_val = round(504*val_ratio)-Cvd_val
    print(noCvd_val)
    
    DownNoCvd=0
    DownCvd=0
    UpNoCvd = fold_NoCvd_total
    UpCvd = fold_Cvd_total

    for i in range(cv):
        X_test = []
        y_test = []
        X_train = []
        y_train = []
        X_val = []
        y_val = []
#         print("============",i,"==============")
        DownNoCvd, UpNoCvd = find_NoCvd_limitsArray(i, DownNoCvd, UpNoCvd, fold_NoCvd_total, fold_NoCvd_residue)
        DownCvd, UpCvd = find_Cvd_limitsArray(i, DownCvd, UpCvd, fold_Cvd_total, fold_Cvd_residue,cv)
#         print(DownNoCvd,UpNoCvd,DownCvd,UpCvd)
        testSubset_total = find_testValSubset(DownNoCvd,UpNoCvd,DownCvd,UpCvd)
        if i!=9:
#             print(UpNoCvd,UpNoCvd+noCvd_val,UpCvd,UpCvd+Cvd_val)
            validationSubset_total = find_testValSubset(UpNoCvd,UpNoCvd+noCvd_val,UpCvd,UpCvd+Cvd_val)
#             print(DownNoCvd,UpNoCvd+noCvd_val,DownCvd,UpCvd+Cvd_val)
            trainSubset_NoCvd ,trainSubset_Cvd ,trainSubset_total = find_trainSubset(DownNoCvd,UpNoCvd+noCvd_val,DownCvd,UpCvd+Cvd_val)
        else:
#             print(DownNoCvd-noCvd_val,DownNoCvd,DownCvd-Cvd_val,DownCvd)
            validationSubset_total = find_testValSubset(DownNoCvd-noCvd_val,DownNoCvd,DownCvd-Cvd_val,DownCvd)
#             print(DownNoCvd-noCvd_val,UpNoCvd,DownCvd-Cvd_val,UpCvd)
            trainSubset_NoCvd ,trainSubset_Cvd ,trainSubset_total = find_trainSubset(DownNoCvd-noCvd_val,UpNoCvd,DownCvd-Cvd_val,UpCvd)
    #creating X_train, y_train, X_test, y_test
        X_test.append(np.delete(testSubset_total, 16, axis=1))
        y_test_temp = np.delete(testSubset_total, slice(0, 16), axis=1)
        y_test.append(np.reshape(y_test_temp, len(y_test_temp)))
        X_test_temp = np.array(X_test)
        X_test = X_test_temp[0]
        y_test_temp = np.array(y_test)
        y_test = y_test_temp[0]
        
        X_val.append(np.delete(validationSubset_total, 16, axis=1))
        y_val_temp = np.delete(validationSubset_total, slice(0, 16), axis=1)
        y_val.append(np.reshape(y_val_temp, len(y_val_temp)))
        X_val_temp = np.array(X_val)
        X_val = X_val_temp[0]
        y_val_temp = np.array(y_val)
        y_val = y_val_temp[0]
        
        X_train.append(np.delete(trainSubset_total, 16, axis=1))
        y_train_temp = np.delete(trainSubset_total, slice(0, 16), axis=1)
        y_train.append(np.reshape(y_train_temp, len(y_train_temp)))
        X_train_temp = np.array(X_train)
        X_train = X_train_temp[0]
        y_train_temp = np.array(y_train)
        y_train = y_train_temp[0]
        
    #add every subset in a list so we can handle thm later 
        test_total.append([X_test,y_test])
        validation_total.append([X_val,y_val])
        train_total.append([X_train,y_train]) 
        train_total_NoCvd.append(trainSubset_NoCvd)
        train_total_Cvd.append(trainSubset_Cvd)
    return train_total, test_total, validation_total, train_total_NoCvd, train_total_Cvd

In [5]:
train_total, test_total, validation_total, train_total_NoCvd, train_total_Cvd = create_train_test_sets(10,dataNoCvd,dataCvd,0.1)

fold_Cvd_total  : 4  fold_Cvd_residue  : 1
fold_NoCvd_total: 51 fold_NoCvd_residue: 9
46


In [6]:
# Chech ratio in each train and test set
def find_ratio(index_list):
    one = 0
    lenght=len(index_list[0])
    for i in range(lenght):
#         print(index_list[1])
        if index_list[1][i] == 1.0 :
            one+= 1
    ratio = one/lenght
    return ratio

for i in range(10):
    print(len(train_total[i][0]))
    ratio = find_ratio(train_total[i])
    print("ratio in train set:", ratio)
    ratio = find_ratio(test_total[i])
    print("ratio in test set:", ratio)
    ratio = find_ratio(validation_total[i])
    print("ratio in validation set:", ratio)

454
ratio in train set: 0.07268722466960352
ratio in test set: 0.07142857142857142
ratio in validation set: 0.08
454
ratio in train set: 0.07268722466960352
ratio in test set: 0.07142857142857142
ratio in validation set: 0.08
454
ratio in train set: 0.07268722466960352
ratio in test set: 0.07142857142857142
ratio in validation set: 0.08
454
ratio in train set: 0.07268722466960352
ratio in test set: 0.07142857142857142
ratio in validation set: 0.08
454
ratio in train set: 0.07268722466960352
ratio in test set: 0.07142857142857142
ratio in validation set: 0.08
454
ratio in train set: 0.07268722466960352
ratio in test set: 0.07142857142857142
ratio in validation set: 0.08
454
ratio in train set: 0.07268722466960352
ratio in test set: 0.07142857142857142
ratio in validation set: 0.08
454
ratio in train set: 0.07268722466960352
ratio in test set: 0.07142857142857142
ratio in validation set: 0.08
454
ratio in train set: 0.07268722466960352
ratio in test set: 0.07142857142857142
ratio in vali

In [7]:
# https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6

from sklearn.metrics import  confusion_matrix,roc_curve, roc_auc_score, accuracy_score
from imxgboost.imbalance_xgb import imbalance_xgboost as imb_xgb
from sklearn.model_selection import StratifiedKFold # import KFold
from sklearn import metrics
from xgboost import XGBClassifier


def my_cross_val( cv, train_total, test_total, validation_total, train_total_NoCvd, train_total_Cvd,params):
    accuracy = []
    specificity = []
    sensitivity = []
    auc = []
    
    # print(X)
    for i in range(cv):
#         X_train = train_total[i][0]
#         y_train = train_total[i][1]
        X_test = test_total[i][0]
        y_test = test_total[i][1]
        
        classifier = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=params[5],
                  colsample_bynode=1, colsample_bytree=params[8], gamma=params[4],
                  learning_rate=params[0], max_delta_step=0, max_depth=params[1],
                  min_child_weight=params[3], missing=None, n_estimators=params[2], n_jobs=1,
                  nthread=None, objective='binary:logistic', random_state=0,
                  reg_alpha=params[9], reg_lambda=params[6], scale_pos_weight=params[7], seed=None,
                  silent=None, subsample=params[10], verbosity=1)
        
        X_train = train_total[i][0]
        y_train = train_total[i][1]
        
        X_val = validation_total[i][0]
        y_val = validation_total[i][1]
        
        eval_set  = [(X_train,y_train), (X_val,y_val)]
        
        classifier.fit( X_train, y_train, eval_set=eval_set, eval_metric="auc", early_stopping_rounds=50,verbose = False )

        
        
        y_pred = classifier.predict(X_test)
        
        cm = confusion_matrix(y_test, y_pred)
    #     print(cm)

        total1=sum(sum(cm))
        #####from confusion matrix calculate accuracy
        accuracy1=(cm[0,0]+cm[1,1])/total1
    #   print ('Accuracy : ', accuracy1)

        specificity1 = cm[0,0]/(cm[0,0]+cm[0,1])
    #   print('Specificity : ', specificity1 )

        sensitivity1 = cm[1,1]/(cm[1,0]+cm[1,1])
    #   print('Sensitivity : ', sensitivity1)
        
#         y = np.array(y_test)
#         pred = np.array(y_pred)
#         fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=2)
#         auc1 = metrics.auc(fpr, tpr)
        
        auc1 = roc_auc_score(y_test, y_pred)

    
        accuracy.append(accuracy1)
        specificity.append(specificity1)
        sensitivity.append(sensitivity1) 
        auc.append(auc1)
    return accuracy, specificity, sensitivity, auc

In [10]:
comb = compute_comb(params)
print(comb[0])
accuracy, specificity, sensitivity, auc = my_cross_val( 10, train_total, test_total, train_total_NoCvd, train_total_Cvd,comb[0])

NameError: name 'compute_comb' is not defined

In [11]:
from statistics import mean
print(mean(accuracy))
print(mean(specificity))
print(mean(sensitivity))
print(mean(auc))

NameError: name 'accuracy' is not defined

In [8]:
# https://www.geeksforgeeks.org/python-all-possible-permutations-of-n-lists/

def compute_comb(params):
    
    # initializing lists 
    list1 = params.get("learning_rate")
    list2 = params.get("max_depth")
    list3 = params.get("n_estimators") 
    list4 = params.get("min_child_weight")
    list5 = params.get("gamma")    
    list6 = params.get("colsample_bylevel")
    list7 = params.get("reg_lambda") 
    list8 = params.get("scale_pos_weight")
    list9 = params.get("colsample_bytree")
    list10 = params.get("reg_alpha")
    list11 = params.get("subsample")
    

#     # printing lists  
#     print ("The original lists are : " + str(list1) +
#                                    " " + str(list2) + 
#                                    " " + str(list3) + 
#                                    " " + str(list4) + 
#                                    " " + str(list5) + 
#                                    " " + str(list6) +
#                                    " " + str(list7) +
#                                    " " + str(list8)) 

    # using list comprehension  
    # to compute all possible permutations 
    res = [[i, j, k, l, m, n, o, p,q,r,s] for i in list1  
                                          for j in list2 
                                          for k in list3 
                                          for l in list4 
                                          for m in list5 
                                          for n in list6 
                                          for o in list7
                                          for p in list8
                                          for q in list9
                                          for r in list10
                                          for s in list11] 

    # printing result 
#     print ("All possible permutations are : " +  str(res))
    return res

In [9]:
class style:
    BOLD = '\033[1m'
    END = '\033[0m'

In [10]:
# https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6

from sklearn.metrics import  confusion_matrix,roc_curve, roc_auc_score, accuracy_score
from imxgboost.imbalance_xgb import imbalance_xgboost as imb_xgb
from sklearn.model_selection import StratifiedKFold # import KFold
from statistics import mean

def gridsearch(params):
    max_sens = 0
    max_spes = 0
    max_acc = 0
    best_params = []
    max_params = []
    max_auc = 0
    combs = compute_comb(params)
    print(len(combs))
    for j in range(len(combs)):
        print (j, end="\r")
        accuracy, specificity, sensitivity, auc = my_cross_val( 10, train_total, test_total, validation_total, train_total_NoCvd, train_total_Cvd,combs[j])
        if mean(auc)>0.65:
            print(style.BOLD + "----------------------------------------------------------------"+ style.END)
            print(combs[j])
            print (style.BOLD + 'accuracy    ' + style.END, mean(accuracy))
            print (style.BOLD + 'specificity ' + style.END, mean(specificity))
            print (style.BOLD + 'sensitivity ' + style.END, mean(sensitivity))
            print (style.BOLD + 'auc         ' + style.END, mean(auc))
            print(style.BOLD + "----------------------------------------------------------------"+ style.END)
#         else:
#             print(combs[j])
#             print("accuracy    ",mean(accuracy))
#             print("specificity ",mean(specificity))
#             print("sensitivity ",mean(sensitivity))
#             print("auc         ",mean(auc))
            
        best_params.append([combs[j],mean(accuracy),mean(specificity),mean(sensitivity),mean(auc)])
        if mean(auc)> max_auc:
            max_sens = mean(sensitivity)
            max_spes = mean(specificity)
            max_acc = mean(accuracy)
            max_auc = mean(auc)
            max_params = combs[j]
    return max_sens, max_params, max_spes, max_acc, max_auc, best_params

# Ratio = 2

 Αποτέλεσμα της gridsearch με τις ακόλουθες παραμέτρους, γίνονται print όλα και αυτά που ικανοποιούν την συνθήκη : if mean(sensitivity)> 0.5 and mean(specificity)>0.6 and mean(accuracy)>0.6 and mean(auc)>0.5 είναι με bold και μεταξύ γραμμών.

In [39]:
# https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/        

params = {
                    "learning_rate"    :[0.3],  #0.01-0.2 Makes the model more robust by shrinking the weights on each step
                    "max_depth"        :[2,3],  #3-10 control over-fitting as higher depth will allow model to learn relations very specific to a particular sample
                    "n_estimators"     :[1000],
                    "min_child_weight" :[1,2,3,4],         #0.5-1 small values might lead to under-fitting
                    "gamma"            :[0,0.5,1],            #Makes the algorithm conservative --> No overfitting
                    "colsample_bylevel" :[0.5,0.75,1], #0.5-1
                     "reg_lambda"      :[1, 1.5, 2], #  it should be explored to reduce overfitting.
                    "scale_pos_weight" :[3,4,5],
                    "colsample_bytree" :[0.5,0.75,1.0],
                    "reg_alpha"        :[0, 0.05,0.1,0.2],
                    "subsample"        :[0.8,1.0]
    
        }

In [40]:
max_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params)

In [41]:
print(max_params)
print(max_acc)
print(max_spes)
print(max_sens)
print(max_auc)

[0.3, 2, 1000, 1, 0, 0.75, 1.5, 5, 1.0, 0.2, 1.0]
0.925
0.9865384615384615
0.145
0.5657692307692308


In [47]:
#Τυπώνονται περιπτώσεις που accuracy>0.6 και specificity>0.6 και sensitivity>0.6 και auc>0.6
s = 0
for i in range(len(best_params)):
    if best_params[i][4] > 0.56:
        print("parameters :",best_params[i][0])
        print("accuracy :",best_params[i][1])
        print("spes     :",best_params[i][2])
        print("sens     :",best_params[i][3])
        print("auc      :",best_params[i][4])
        print("------------------------------------------------")
        s+=1
print(s)

parameters : [0.3, 2, 1000, 1, 0, 0.75, 1.5, 5, 1.0, 0.2, 1.0]
accuracy : 0.925
spes     : 0.9865384615384615
sens     : 0.145
auc      : 0.5657692307692308
------------------------------------------------
parameters : [0.3, 2, 1000, 1, 0.5, 0.75, 1.5, 5, 1.0, 0.2, 1.0]
accuracy : 0.925
spes     : 0.9865384615384615
sens     : 0.145
auc      : 0.5657692307692308
------------------------------------------------
parameters : [0.3, 2, 1000, 1, 1, 0.75, 1.5, 5, 1.0, 0.2, 1.0]
accuracy : 0.925
spes     : 0.9865384615384615
sens     : 0.145
auc      : 0.5657692307692308
------------------------------------------------
parameters : [0.3, 3, 1000, 1, 1, 0.5, 2, 5, 1.0, 0.2, 1.0]
accuracy : 0.9035714285714286
spes     : 0.9614253393665159
sens     : 0.17
auc      : 0.565712669683258
------------------------------------------------
parameters : [0.3, 3, 1000, 3, 1, 0.75, 2, 5, 1.0, 0.2, 0.8]
accuracy : 0.8982142857142857
spes     : 0.9557315233785821
sens     : 0.17
auc      : 0.562865761689291


In [27]:

params = {
                    "learning_rate"    :[0.3],  #0.01-0.2 Makes the model more robust by shrinking the weights on each step
                    "max_depth"        :[2,3,4],  #3-10 control over-fitting as higher depth will allow model to learn relations very specific to a particular sample
                    "n_estimators"     :[1000],
                    "min_child_weight" :[1,2,3,4],         #0.5-1 small values might lead to under-fitting
                    "gamma"            :[0,0.5,1],            #Makes the algorithm conservative --> No overfitting
                    "colsample_bylevel" :[0.5,0.75,1], #0.5-1
                     "reg_lambda"      :[1, 1.5, 2], #  it should be explored to reduce overfitting.
                    "scale_pos_weight" :[6,7,8],
                    "colsample_bytree" :[0.5,0.75,1.0],
                    "reg_alpha"        :[0, 0.05,0.1,0.2,0.3],
                    "subsample"        :[0.8,0.9,1.0]
    
        }

In [28]:
max_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params)

43740


In [29]:
print(max_params)
print(max_acc)
print(max_spes)
print(max_sens)
print(max_auc)

[0.3, 3, 1000, 4, 1, 0.5, 1.5, 7, 0.5, 0.05, 0.8]
0.8678571428571429
0.9114630467571644
0.32
0.6157315233785822


In [30]:
#Τυπώνονται περιπτώσεις που accuracy>0.6 και specificity>0.6 και sensitivity>0.6 και auc>0.6
s = 0
for i in range(len(best_params)):
    if best_params[i][4] > 0.61:
        print("parameters :",best_params[i][0])
        print("accuracy :",best_params[i][1])
        print("spes     :",best_params[i][2])
        print("sens     :",best_params[i][3])
        print("auc      :",best_params[i][4])
        print("------------------------------------------------")
        s+=1
print(s)

parameters : [0.3, 3, 1000, 3, 0.5, 0.5, 1, 6, 0.5, 0, 0.8]
accuracy : 0.8875
spes     : 0.9345776772247361
sens     : 0.295
auc      : 0.614788838612368
------------------------------------------------
parameters : [0.3, 3, 1000, 4, 0, 0.5, 1.5, 7, 0.5, 0.1, 0.8]
accuracy : 0.8625
spes     : 0.9057315233785822
sens     : 0.32
auc      : 0.6128657616892911
------------------------------------------------
parameters : [0.3, 3, 1000, 4, 1, 0.5, 1.5, 7, 0.5, 0.05, 0.8]
accuracy : 0.8678571428571429
spes     : 0.9114630467571644
sens     : 0.32
auc      : 0.6157315233785822
------------------------------------------------
3


In [34]:

params = {
                    "learning_rate"    :[0.3],  #0.01-0.2 Makes the model more robust by shrinking the weights on each step
                    "max_depth"        :[2,3,4],  #3-10 control over-fitting as higher depth will allow model to learn relations very specific to a particular sample
                    "n_estimators"     :[1000],
                    "min_child_weight" :[5,6],         #0.5-1 small values might lead to under-fitting
                    "gamma"            :[0,0.5,1],            #Makes the algorithm conservative --> No overfitting
                    "colsample_bylevel" :[0.5,0.75,1], #0.5-1
                     "reg_lambda"      :[1, 1.5, 2], #  it should be explored to reduce overfitting.
                    "scale_pos_weight" :[6,7,8],
                    "colsample_bytree" :[0.5,0.75,1.0],
                    "reg_alpha"        :[0, 0.05,0.1,0.2,0.3],
                    "subsample"        :[0.8,0.9,1.0]
    
        }


In [35]:
max_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params)

21870


In [37]:
print(max_params)
print(max_acc)
print(max_spes)
print(max_sens)
print(max_auc)

[0.3, 3, 1000, 5, 1, 0.5, 2, 8, 1.0, 0.2, 1.0]
0.8642857142857143
0.9076168929110106
0.315
0.6113084464555053


In [38]:
#Τυπώνονται περιπτώσεις που accuracy>0.6 και specificity>0.6 και sensitivity>0.6 και auc>0.6
s = 0
for i in range(len(best_params)):
    if best_params[i][4] > 0.61:
        print("parameters :",best_params[i][0])
        print("accuracy :",best_params[i][1])
        print("spes     :",best_params[i][2])
        print("sens     :",best_params[i][3])
        print("auc      :",best_params[i][4])
        print("------------------------------------------------")
        s+=1
print(s)

parameters : [0.3, 3, 1000, 5, 1, 0.5, 2, 8, 1.0, 0.2, 1.0]
accuracy : 0.8642857142857143
spes     : 0.9076168929110106
sens     : 0.315
auc      : 0.6113084464555053
------------------------------------------------
1


In [39]:
# https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/        

params = {
                    "learning_rate"    :[0.1],  #0.01-0.2 Makes the model more robust by shrinking the weights on each step
                    "max_depth"        :[2,3,4,5],  #3-10 control over-fitting as higher depth will allow model to learn relations very specific to a particular sample
                    "n_estimators"     :[1000],
                    "min_child_weight" :[1,2,3,4],         #0.5-1 small values might lead to under-fitting
                    "gamma"            :[0,0.5,1],            #Makes the algorithm conservative --> No overfitting
                    "colsample_bylevel" :[0.5,0.75,1], #0.5-1
                     "reg_lambda"      :[1, 1.5, 2], #  it should be explored to reduce overfitting.
                    "scale_pos_weight" :[3,4,5,6],
                    "colsample_bytree" :[0.5,0.75,1.0],
                    "reg_alpha"        :[0, 0.05,0.1,0.2],
                    "subsample"        :[0.8,0.9,1.0]
    
        }

In [50]:
max_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params)

62208


In [51]:
print(max_params)
print(max_acc)
print(max_spes)
print(max_sens)
print(max_auc)

[0.1, 4, 1000, 1, 1, 0.5, 1, 6, 0.75, 0.2, 0.9]
0.8892857142857142
0.9403469079939668
0.245
0.5926734539969833


In [52]:
#Τυπώνονται περιπτώσεις που accuracy>0.6 και specificity>0.6 και sensitivity>0.6 και auc>0.6
s = 0
for i in range(len(best_params)):
    if best_params[i][4] > 0.59:
        print("parameters :",best_params[i][0])
        print("accuracy :",best_params[i][1])
        print("spes     :",best_params[i][2])
        print("sens     :",best_params[i][3])
        print("auc      :",best_params[i][4])
        print("------------------------------------------------")
        s+=1
print(s)

parameters : [0.1, 4, 1000, 1, 1, 0.5, 1, 6, 0.75, 0, 0.9]
accuracy : 0.8857142857142857
spes     : 0.9364253393665158
sens     : 0.245
auc      : 0.5907126696832579
------------------------------------------------
parameters : [0.1, 4, 1000, 1, 1, 0.5, 1, 6, 0.75, 0.2, 0.9]
accuracy : 0.8892857142857142
spes     : 0.9403469079939668
sens     : 0.245
auc      : 0.5926734539969833
------------------------------------------------
2


In [32]:
# https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/        

params = {
                    "learning_rate"    :[0.1],  #0.01-0.2 Makes the model more robust by shrinking the weights on each step
                    "max_depth"        :[3,4,5,6],  #3-10 control over-fitting as higher depth will allow model to learn relations very specific to a particular sample
                    "n_estimators"     :[1000],
                    "min_child_weight" :[1,2,3,4,5,6],         #0.5-1 small values might lead to under-fitting
                    "gamma"            :[0,0.5,1],            #Makes the algorithm conservative --> No overfitting
                    "colsample_bylevel" :[0.5,0.75,1], #0.5-1
                     "reg_lambda"      :[1, 1.5, 2], #  it should be explored to reduce overfitting.
                    "scale_pos_weight" :[7,8,9],
                    "colsample_bytree" :[0.5,0.75,1.0],
                    "reg_alpha"        :[0, 0.05,0.1,0.2],
                    "subsample"        :[0.8,0.9,1.0]
    
        }

In [33]:
max_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params)

69984


In [57]:
# https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/        

params = {
                    "learning_rate"    :[0.4],  #0.01-0.2 Makes the model more robust by shrinking the weights on each step
                    "max_depth"        :[2,3,4,5],  #3-10 control over-fitting as higher depth will allow model to learn relations very specific to a particular sample
                    "n_estimators"     :[1000],
                    "min_child_weight" :[1,2,3,4],         #0.5-1 small values might lead to under-fitting
                    "gamma"            :[0,0.5,1],            #Makes the algorithm conservative --> No overfitting
                    "colsample_bylevel" :[0.5,0.75,1], #0.5-1
                     "reg_lambda"      :[1, 1.5, 2], #  it should be explored to reduce overfitting.
                    "scale_pos_weight" :[4,5,6,7,8,9],
                    "colsample_bytree" :[0.5,0.75,1.0],
                    "reg_alpha"        :[0, 0.05,0.1,0.2],
                    "subsample"        :[0.8,0.9,1.0]
    
        }

In [58]:
max_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params)

93312


In [59]:
print(max_params)
print(max_acc)
print(max_spes)
print(max_sens)
print(max_auc)

[0.4, 3, 1000, 4, 0.5, 0.5, 1, 6, 0.5, 0.2, 0.8]
0.8696428571428572
0.9132730015082956
0.315
0.6141365007541478


In [62]:
#Τυπώνονται περιπτώσεις που accuracy>0.6 και specificity>0.6 και sensitivity>0.6 και auc>0.6
s = 0
for i in range(len(best_params)):
    if best_params[i][4] > 0.61:
        print("parameters :",best_params[i][0])
        print("accuracy :",best_params[i][1])
        print("spes     :",best_params[i][2])
        print("sens     :",best_params[i][3])
        print("auc      :",best_params[i][4])
        print("------------------------------------------------")
        s+=1
print(s)

parameters : [0.4, 3, 1000, 4, 0.5, 0.5, 1, 6, 0.5, 0.2, 0.8]
accuracy : 0.8696428571428572
spes     : 0.9132730015082956
sens     : 0.315
auc      : 0.6141365007541478
------------------------------------------------
1


In [63]:
# https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/        

params = {
                    "learning_rate"    :[0.4],  #0.01-0.2 Makes the model more robust by shrinking the weights on each step
                    "max_depth"        :[2,3,4,5],  #3-10 control over-fitting as higher depth will allow model to learn relations very specific to a particular sample
                    "n_estimators"     :[1000],
                    "min_child_weight" :[5,6],         #0.5-1 small values might lead to under-fitting
                    "gamma"            :[0,0.5,1],            #Makes the algorithm conservative --> No overfitting
                    "colsample_bylevel" :[0.5,0.75,1], #0.5-1
                     "reg_lambda"      :[1, 1.5, 2], #  it should be explored to reduce overfitting.
                    "scale_pos_weight" :[4,5,6,7,8,9],
                    "colsample_bytree" :[0.5,0.75,1.0],
                    "reg_alpha"        :[0,0.1,0.2,0.3],
                    "subsample"        :[0.8,0.9,1.0]
    
        }
# https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/        


In [ ]:
max_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params)

46656


In [67]:
print(max_params)
print(max_acc)
print(max_spes)
print(max_sens)
print(max_auc)

[0.4, 5, 1000, 5, 1, 0.5, 1.5, 8, 1.0, 0.2, 1.0]
0.8660714285714286
0.9095022624434389
0.32
0.6147511312217194


In [68]:
#Τυπώνονται περιπτώσεις που accuracy>0.6 και specificity>0.6 και sensitivity>0.6 και auc>0.6
s = 0
for i in range(len(best_params)):
    if best_params[i][4] > 0.61:
        print("parameters :",best_params[i][0])
        print("accuracy :",best_params[i][1])
        print("spes     :",best_params[i][2])
        print("sens     :",best_params[i][3])
        print("auc      :",best_params[i][4])
        print("------------------------------------------------")
        s+=1
print(s)

parameters : [0.4, 3, 1000, 5, 0, 0.5, 2, 9, 1.0, 0.1, 1.0]
accuracy : 0.8160714285714286
spes     : 0.8518476621417798
sens     : 0.37
auc      : 0.6109238310708899
------------------------------------------------
parameters : [0.4, 3, 1000, 5, 0.5, 0.5, 2, 9, 1.0, 0, 1.0]
accuracy : 0.8410714285714286
spes     : 0.8806938159879336
sens     : 0.345
auc      : 0.6128469079939668
------------------------------------------------
parameters : [0.4, 3, 1000, 5, 0.5, 0.5, 2, 9, 1.0, 0.1, 1.0]
accuracy : 0.8214285714285714
spes     : 0.8576168929110105
sens     : 0.37
auc      : 0.6138084464555053
------------------------------------------------
parameters : [0.4, 5, 1000, 5, 0, 0.5, 1.5, 6, 1.0, 0.3, 0.9]
accuracy : 0.8803571428571428
spes     : 0.9268853695324284
sens     : 0.295
auc      : 0.6109426847662142
------------------------------------------------
parameters : [0.4, 5, 1000, 5, 1, 0.5, 1.5, 8, 1.0, 0.2, 1.0]
accuracy : 0.8660714285714286
spes     : 0.9095022624434389
sens     : 0

In [22]:
# https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/        

params = {
                    "learning_rate"    :[0.4],  #0.01-0.2 Makes the model more robust by shrinking the weights on each step
                    "max_depth"        :[6,7],  #3-10 control over-fitting as higher depth will allow model to learn relations very specific to a particular sample
                    "n_estimators"     :[1000],
                    "min_child_weight" :[5,6],         #0.5-1 small values might lead to under-fitting
                    "gamma"            :[0,0.5,1],            #Makes the algorithm conservative --> No overfitting
                    "colsample_bylevel" :[0.5,0.75,1], #0.5-1
                     "reg_lambda"      :[1, 1.5, 2], #  it should be explored to reduce overfitting.
                    "scale_pos_weight" :[4,5,6,7,8,9],
                    "colsample_bytree" :[0.5,0.75,1.0],
                    "reg_alpha"        :[0,0.1,0.2,0.3],
                    "subsample"        :[0.8,0.9,1.0]
    
        }


In [23]:
max_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params)

23328


In [24]:
print(max_params)
print(max_acc)
print(max_spes)
print(max_sens)
print(max_auc)

[0.4, 6, 1000, 6, 0.5, 0.5, 1, 6, 1.0, 0, 0.9]
0.8821428571428571
0.9306561085972851
0.27
0.6003280542986426


In [25]:
#Τυπώνονται περιπτώσεις που accuracy>0.6 και specificity>0.6 και sensitivity>0.6 και auc>0.6
s = 0
for i in range(len(best_params)):
    if best_params[i][4] > 0.6:
        print("parameters :",best_params[i][0])
        print("accuracy :",best_params[i][1])
        print("spes     :",best_params[i][2])
        print("sens     :",best_params[i][3])
        print("auc      :",best_params[i][4])
        print("------------------------------------------------")
        s+=1
print(s)

parameters : [0.4, 6, 1000, 6, 0.5, 0.5, 1, 6, 1.0, 0, 0.9]
accuracy : 0.8821428571428571
spes     : 0.9306561085972851
sens     : 0.27
auc      : 0.6003280542986426
------------------------------------------------
1


In [11]:
# https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/        

params = {
                    "learning_rate"    :[0.2],  #0.01-0.2 Makes the model more robust by shrinking the weights on each step
                    "max_depth"        :[2,3,4,5],  #3-10 control over-fitting as higher depth will allow model to learn relations very specific to a particular sample
                    "n_estimators"     :[1000],
                    "min_child_weight" :[1,2,3,4],         #0.5-1 small values might lead to under-fitting
                    "gamma"            :[0,0.5,1],            #Makes the algorithm conservative --> No overfitting
                    "colsample_bylevel" :[0.5,0.75,1], #0.5-1
                     "reg_lambda"      :[1, 1.5, 2], #  it should be explored to reduce overfitting.
                    "scale_pos_weight" :[4,5,6,7,8,9],
                    "colsample_bytree" :[0.5,0.75,1.0],
                    "reg_alpha"        :[0, 0.05,0.1,0.2],
                    "subsample"        :[0.8,0.9,1.0]
    
        }

In [12]:
max_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params)

93312


In [13]:
print(max_params)
print(max_acc)
print(max_spes)
print(max_sens)
print(max_auc)

[0.2, 3, 1000, 1, 0.5, 0.75, 1, 9, 0.5, 0.05, 0.8]
0.8517857142857143
0.8901960784313725
0.365
0.6275980392156862


In [14]:
#Τυπώνονται περιπτώσεις που accuracy>0.6 και specificity>0.6 και sensitivity>0.6 και auc>0.6
s = 0
for i in range(len(best_params)):
    if best_params[i][4] > 0.62:
        print("parameters :",best_params[i][0])
        print("accuracy :",best_params[i][1])
        print("spes     :",best_params[i][2])
        print("sens     :",best_params[i][3])
        print("auc      :",best_params[i][4])
        print("------------------------------------------------")
        s+=1
print(s)

parameters : [0.2, 3, 1000, 1, 0.5, 0.75, 1, 9, 0.5, 0.05, 0.8]
accuracy : 0.8517857142857143
spes     : 0.8901960784313725
sens     : 0.365
auc      : 0.6275980392156862
------------------------------------------------
parameters : [0.2, 3, 1000, 1, 1, 0.75, 1, 9, 0.5, 0, 0.8]
accuracy : 0.8464285714285714
spes     : 0.8844268476621417
sens     : 0.365
auc      : 0.6247134238310709
------------------------------------------------
2


In [16]:
# https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/        

params = {
                    "learning_rate"    :[0.2],  #0.01-0.2 Makes the model more robust by shrinking the weights on each step
                    "max_depth"        :[2,3,4,5],  #3-10 control over-fitting as higher depth will allow model to learn relations very specific to a particular sample
                    "n_estimators"     :[1000],
                    "min_child_weight" :[1,2,3,4],         #0.5-1 small values might lead to under-fitting
                    "gamma"            :[0,0.5,1],            #Makes the algorithm conservative --> No overfitting
                    "colsample_bylevel" :[0.5,0.75,1], #0.5-1
                     "reg_lambda"      :[1, 1.5, 2], #  it should be explored to reduce overfitting.
                    "scale_pos_weight" :[10,11,12],
                    "colsample_bytree" :[0.5,0.75,1.0],
                    "reg_alpha"        :[0, 0.05,0.1,0.2],
                    "subsample"        :[0.8,0.9,1.0]
    
        }

In [17]:
max_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params)

46656


In [18]:
print(max_params)
print(max_acc)
print(max_spes)
print(max_sens)
print(max_auc)

[0.2, 4, 1000, 2, 0.5, 0.5, 2, 10, 1.0, 0.05, 0.9]
0.8482142857142857
0.8845022624434389
0.395
0.6397511312217194


In [19]:
#Τυπώνονται περιπτώσεις που accuracy>0.6 και specificity>0.6 και sensitivity>0.6 και auc>0.6
s = 0
for i in range(len(best_params)):
    if best_params[i][4] > 0.63:
        print("parameters :",best_params[i][0])
        print("accuracy :",best_params[i][1])
        print("spes     :",best_params[i][2])
        print("sens     :",best_params[i][3])
        print("auc      :",best_params[i][4])
        print("------------------------------------------------")
        s+=1
print(s)

parameters : [0.2, 3, 1000, 1, 1, 1, 1.5, 12, 0.75, 0, 0.9]
accuracy : 0.8089285714285714
spes     : 0.840158371040724
sens     : 0.42
auc      : 0.630079185520362
------------------------------------------------
parameters : [0.2, 4, 1000, 2, 0.5, 0.5, 2, 10, 1.0, 0, 0.9]
accuracy : 0.8553571428571429
spes     : 0.8941553544494721
sens     : 0.37
auc      : 0.6320776772247361
------------------------------------------------
parameters : [0.2, 4, 1000, 2, 0.5, 0.5, 2, 10, 1.0, 0.05, 0.9]
accuracy : 0.8482142857142857
spes     : 0.8845022624434389
sens     : 0.395
auc      : 0.6397511312217194
------------------------------------------------
parameters : [0.2, 4, 1000, 2, 0.5, 0.5, 2, 10, 1.0, 0.2, 0.9]
accuracy : 0.8571428571428572
spes     : 0.8960784313725491
sens     : 0.37
auc      : 0.6330392156862745
------------------------------------------------
parameters : [0.2, 4, 1000, 2, 1, 0.5, 2, 10, 1.0, 0, 0.9]
accuracy : 0.8535714285714285
spes     : 0.8921945701357467
sens     : 0.3

In [ ]:
parameters : [0.2, 4, 1000, 2, 0.5, 0.5, 2, 10, 1.0, 0.05, 0.9]
accuracy : 0.8482142857142857
spes     : 0.8845022624434389
sens     : 0.395
auc      : 0.6397511312217194
    
parameters : [0.2, 3, 1000, 1, 0.5, 0.75, 1, 9, 0.5, 0.05, 0.8]
accuracy : 0.8517857142857143
spes     : 0.8901960784313725
sens     : 0.365
auc      : 0.6275980392156862
    
parameters : [0.2, 4, 1000, 2, 0.5, 0.5, 2, 10, 1.0, 0.2, 0.9]
accuracy : 0.8571428571428572
spes     : 0.8960784313725491
sens     : 0.37
auc      : 0.6330392156862745

In [ ]:

params = {
                    "learning_rate"    :[0.4],  
                    "max_depth"        :[2,3,4,5,6,7],  
                    "n_estimators"     :[1000],
                    "min_child_weight" :[1,2,3,4,5,6],         
                    "gamma"            :[0,0.5,1],           
                    "colsample_bylevel" :[0.5,0.75,1], 
                     "reg_lambda"      :[1, 1.5, 2], 
                    "scale_pos_weight" :[4,5,6,7,8,9],
                    "colsample_bytree" :[0.5,0.75,1.0],
                    "reg_alpha"        :[0, 0.05,0.1,0.2,0.3],
                    "subsample"        :[0.8,0.9,1.0]
    
        }

params = {
                    "learning_rate"    :[0.3], 
                    "max_depth"        :[2,3,4],  
                    "n_estimators"     :[1000],
                    "min_child_weight" :[1,2,3,4,5,6],         
                    "gamma"            :[0,0.5,1],            
                    "colsample_bylevel" :[0.5,0.75,1], 
                     "reg_lambda"      :[1, 1.5, 2], 
                    "scale_pos_weight" :[3,4,5,6,7,8],
                    "colsample_bytree" :[0.5,0.75,1.0],
                    "reg_alpha"        :[0, 0.05,0.1,0.2,0.3],
                    "subsample"        :[0.8,0.9,1.0]
    
        }

params = {
                    "learning_rate"    :[0.2],  
                    "max_depth"        :[2,3,4,5],  
                    "n_estimators"     :[1000],
                    "min_child_weight" :[1,2,3,4],         
                    "gamma"            :[0,0.5,1],            
                    "colsample_bylevel" :[0.5,0.75,1], 
                     "reg_lambda"      :[1, 1.5, 2], 
                    "scale_pos_weight" :[4,5,6,7,8,9,10,11,12],
                    "colsample_bytree" :[0.5,0.75,1.0],
                    "reg_alpha"        :[0, 0.05,0.1,0.2],
                    "subsample"        :[0.8,0.9,1.0]
    
        }
params = {
                    "learning_rate"    :[0.1],  
                    "max_depth"        :[2,3,4,5,6],  
                    "n_estimators"     :[1000],
                    "min_child_weight" :[1,2,3,4,5,6],         
                    "gamma"            :[0,0.5,1],            
                    "colsample_bylevel" :[0.5,0.75,1], 
                     "reg_lambda"      :[1, 1.5, 2], 
                    "scale_pos_weight" :[3,4,5,6,7,8,9],
                    "colsample_bytree" :[0.5,0.75,1.0],
                    "reg_alpha"        :[0, 0.05,0.1,0.2],
                    "subsample"        :[0.8,0.9,1.0]
    
        }